In [11]:
import tensorflow as tf

import pandas as pd

import numpy as np

import time

from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [12]:
iris = load_iris()

iris_X, iris_y = iris.data[:-1,:], iris.target[:-1]

iris_y= pd.get_dummies(iris_y).values

trainX, testX, trainY, testY = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)

print(trainX.shape)

print(testX.shape)

print(trainY.shape)

print(testY.shape)

(99, 4)
(50, 4)
(99, 3)
(50, 3)


In [13]:
# numFeatures是我们输入数据中的特征数量。

# 在iris数据集中，这个数字是'4'

numFeatures = trainX.shape[1]

# numLabels使我们数据集的类的数量

# 在iris数据集中，这个数字是'3'

numLabels = trainY.shape[1]

# 占位符

# 'None' 意味着TensorFlow不应该期望在该维度中有一个固定的数字

X = tf.placeholder(tf.float32, [None, numFeatures]) # Iris 有4个特征，所以X是来保存我们的数据的张量

yGold = tf.placeholder(tf.float32, [None, numLabels]) # 这将是我们的3个类矩阵的正确答案

W = tf.Variable(tf.zeros([4, 3]))  # 4维输入和3个类

b = tf.Variable(tf.zeros([3])) # 3维输出 [0,0,1],[0,1,0],[1,0,0]

#随机抽取标准偏差为.01的正态分布

weights = tf.Variable(tf.random_normal([numFeatures,numLabels],
                                       mean=0,
                                       stddev=0.01,
                                       name="weights"))

bias = tf.Variable(tf.random_normal([1,numLabels],
                                    mean=0,
                                    stddev=0.01,
                                    name="bias"))

In [14]:
# Logistic回归方程的三要素分解

# 注意这些feed到其他

apply_weights_OP = tf.matmul(X, weights, name="apply_weights")

add_bias_OP = tf.add(apply_weights_OP, bias, name="add_bias") 

# 激励函数
# activation_OP = tf.nn.relu(add_bias_OP, name="activation")
# activation_OP = tf.nn.relu6(add_bias_OP, name="activation")
# activation_OP = tf.nn.elu(add_bias_OP, name="activation")
# activation_OP = tf.nn.softplus(add_bias_OP, name="activation")
# activation_OP = tf.nn.softsign(add_bias_OP, name="activation")
# activation_OP = tf.nn.dropout(add_bias_OP, name="activation")
# activation_OP = tf.nn.bias_add(add_bias_OP, name="activation")
activation_OP = tf.nn.tanh(add_bias_OP, name="activation")
# activation_OP = tf.nn.sigmoid(add_bias_OP, name="activation")

In [15]:
# 我们训练中的Epochs数

numEpochs = 700

# 定义我们的学习率迭代 (衰减)

learningRate = tf.train.exponential_decay(learning_rate=0.0008,
                                          global_step= 1,
                                          decay_steps=trainX.shape[0],
                                          decay_rate= 0.95,
                                          staircase=True)

In [16]:
#定义我们的成本函数 - 平方均值误差

cost_OP = tf.nn.l2_loss(activation_OP-yGold, name="squared_error_cost")

#定义我们的渐变下降
# training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)
# training_OP = tf.train.AdadeltaOptimizer(learningRate).minimize(cost_OP)
# training_OP = tf.train.AdagradOptimizerlearningRate).minimize(cost_OP)
# training_OP = tf.train.MomentumOptimizer(learningRate).minimize(cost_OP)
training_OP = tf.train.AdamOptimizer(learningRate).minimize(cost_OP)

In [17]:
# 创建一个tensorflow会话

sess = tf.Session()

# 初始化我们的权重和偏差变量

init_OP = tf.global_variables_initializer()

# 初始化所有tensorflow变量

sess.run(init_OP)

In [18]:
# argmax(activation_OP, 1)以最大概率返回标签

# argmax(yGold, 1)是正确的标签

correct_predictions_OP = tf.equal(tf.argmax(activation_OP,1),tf.argmax(yGold,1))

# 如果每个错误预测为0并且每个真实预测为1，则平均值会返回我们的准确性

accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

# 汇总op的回归输出

activation_summary_OP = tf.summary.histogram("output", activation_OP)

# 汇总op的准确度

accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)

# 汇总op的成本

cost_summary_OP = tf.summary.scalar("cost", cost_OP)

# 汇总检查每次迭代后变量(W, b) 是如何更新

weightSummary = tf.summary.histogram("weights", weights.eval(session=sess))

biasSummary = tf.summary.histogram("biases", bias.eval(session=sess))

# 合并所有的汇总

merged = tf.summary.merge([activation_summary_OP, accuracy_summary_OP, cost_summary_OP, weightSummary, biasSummary])

# 汇总writer

writer = tf.summary.FileWriter("summary_logs", sess.graph)

In [19]:
# 初始化报告变量

cost = 0

diff = 1

epoch_values = []

accuracy_values = []

cost_values = []

# 训练 epochs

for i in range(numEpochs):

    if i > 1 and diff < .0001:

        print("change in cost %g; convergence."%diff)

        break

    else:

        # 运行训练

        step = sess.run(training_OP, feed_dict={X: trainX, yGold: trainY})

        # Report occasional stats

        if i % 100 == 0:

            # 将epoch添加到epoch_values

            epoch_values.append(i)

            # 基于测试集数据生成准确度

            train_accuracy, newCost = sess.run([accuracy_OP, cost_OP], feed_dict={X: trainX, yGold: trainY})

            # 为实时图形变量添加准确性

            accuracy_values.append(train_accuracy)

            # 为实时图形变量添加成本

            cost_values.append(newCost)

            # 对变量重新分配值

            diff = abs(newCost - cost)

            cost = newCost

            #生成输出语句

            print("step %d, training accuracy %g, cost %g, change in cost %g"%(i, train_accuracy, newCost, diff))

# How well do we perform on held-out test data?

print("final accuracy on test set: %s" %str(sess.run(accuracy_OP,
                                                     feed_dict={X: testX,
                                                                yGold: testY})))

step 0, training accuracy 0.313131, cost 49.0531, change in cost 49.0531
step 100, training accuracy 0.686869, cost 25.1193, change in cost 23.9338
step 200, training accuracy 0.676768, cost 21.1163, change in cost 4.003
step 300, training accuracy 0.686869, cost 19.6503, change in cost 1.46595
step 400, training accuracy 0.676768, cost 19.0298, change in cost 0.620533
step 500, training accuracy 0.707071, cost 18.6869, change in cost 0.342886
step 600, training accuracy 0.717172, cost 18.4458, change in cost 0.241039
final accuracy on test set: 0.76
